Steps of the pipeline in this notebook:

1. Load raw data
2. Create preprocessed data
3. Get predictions of a model
4. Dimensionality reduction (U-MAP)
5. Find cluster of interest of FN in dimensionality reduction space
6. Plot the cluster of interest in raw data space

In [1]:
!pip install umap-learn


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import umap
from sklearn.neighbors import KDTree
from tqdm import tqdm

random_seed = 10
color_map = mcolors.ListedColormap(['blue', 'green', 'red', 'orange']) # TP, TN, FP, FN

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. Load raw data

def load_all_partitions(directory):
    all_files = glob.glob(os.path.join(directory, '*.csv'))
    df_list = [pd.read_csv(file) for file in all_files]
    return pd.concat(df_list, ignore_index=True)

x_raw = load_all_partitions('data/AF-Raw-Data/AF Data/all_raw_data_csv')

# Remove rows that are duplicates or have values > 1800
n_before = x_raw.shape[0]
x_raw = x_raw.drop_duplicates()
x_raw = x_raw[(x_raw.T < 1800).all()]
print(f"Removed {n_before - x_raw.shape[0]} rows with values > 1800 or are duplicate")

# Balance the dataset, by taking the same number of samples from each class
class_1 = x_raw[x_raw['Class_Label'] == 1]
class_0 = x_raw[x_raw['Class_Label'] == 0].sample(len(class_1), random_state=random_seed)
x_raw = pd.concat([class_1, class_0])
print(f"Balanced dataset: {x_raw.shape[0]} samples in total")

x_raw.reset_index(drop=True, inplace=True)
x_raw['Sample_id'] = x_raw.index

# Split the class label from the features and split the data into train and test
y_raw = x_raw[['Class_Label', 'Sample_id']]
x_raw = x_raw.drop(columns=['Class_Label'])
x_raw_train, x_raw_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=0.2, random_state=random_seed)


Removed 136070 rows with values > 1800 or are duplicate
Balanced dataset: 109272 samples in total


In [4]:
# 2. Create preprocessed data

# 30 bins (of 50 milliseconds) are created covering R-R intervals of 200 ms up to 1700 ms. For each
#sample the frequency of an R-R interval occurring in a certain bin was counted.

# Function to bin and count intervals for a row
def bin_row(row, bin_edges):
    intervals = row[:-1].values # Exclude the class label from binning
    bin_indices = np.digitize(intervals, bins=bin_edges, right=False)
    bin_counts = np.bincount(bin_indices, minlength=len(bin_edges)+1)[1:-1]  # Exclude counts outside defined bins
    return bin_counts

bin_edges = np.arange(200, 1751, 50) #ms
bin_column_names = [f'bin_{i + 1}' for i in range(len(bin_edges) - 1)]

def preprocess_into_bins(x_data):
    x_bins = x_data.apply(lambda row: bin_row(row, bin_edges), axis=1, result_type='expand')
    x_bins.columns = bin_column_names
    x_bins['Sample_id'] = x_data['Sample_id']
    return x_bins

x_bins_train = preprocess_into_bins(x_raw_train)
x_bins_test = preprocess_into_bins(x_raw_test)
x_bins = pd.concat([x_bins_train, x_bins_test], ignore_index=True)
x_bins = x_bins.sort_values('Sample_id')


In [5]:
# 3. Get predictions of a model

# Normalise using standard scaler
scaler = StandardScaler()
x_bins_train_scaled = scaler.fit_transform(x_bins_train[bin_column_names])
x_bins_test_scaled = scaler.transform(x_bins_test[bin_column_names])

svm = SVC(random_state=random_seed, kernel='rbf')
svm.fit(x_bins_train_scaled, y_train['Class_Label'])

SVC(random_state=10)

In [ ]:
y_pred_test = svm.predict(x_bins_test_scaled)
y_pred_train = svm.predict(x_bins_train_scaled)

acc_test_svm = accuracy_score(y_test['Class_Label'], y_pred_test)
f1_test_svm = f1_score(y_test['Class_Label'], y_pred_test)

acc_train_svm = accuracy_score(y_train['Class_Label'], y_pred_train)
f1_train_svm = f1_score(y_train['Class_Label'], y_pred_train)

print(f"Test accuracy: {round(acc_test_svm, 4)}, F1: {round(f1_test_svm, 4)}")
print(f"Train accuracy: {round(acc_train_svm, 4)}, F1: {round(f1_train_svm, 4)}")

In [ ]:
cm_test = confusion_matrix(y_test['Class_Label'], y_pred_test)
#visualise it nicely
plt.figure(figsize=(2, 2))
plt.imshow(cm_test, cmap='binary')
plt.grid(False)
for i in range(2):
    for j in range(2):
        plt.text(j, i, cm_test[i, j], ha='center', va='center', color='red')
plt.ylabel('True'), plt.xlabel('Predicted'), plt.xticks([0, 1], ['0', '1']), plt.yticks([0, 1], ['0', '1'])
plt.title('Confusion matrix of the test set')
plt.show()

In [ ]:
# To the test set, add the predictions of the model
x_bins_test['Prediction'] = y_pred_test
x_bins_train['Prediction'] = y_pred_train
x_bins_test['Class_Label'] = y_test['Class_Label']
x_bins_train['Class_Label'] = y_train['Class_Label']

def add_col_cm_label(x_df):
    # Add a column indicating TP, TN, FP, FN, name the column 'CM_Label'
    x_df['CM_Label'] = ''
    x_df.loc[(x_df['Prediction'] == 1) & (x_df['Class_Label'] == 1), 'CM_Label'] = 'TP'
    x_df.loc[(x_df['Prediction'] == 0) & (x_df['Class_Label'] == 0), 'CM_Label'] = 'TN'
    x_df.loc[(x_df['Prediction'] == 1) & (x_df['Class_Label'] == 0), 'CM_Label'] = 'FP'
    x_df.loc[(x_df['Prediction'] == 0) & (x_df['Class_Label'] == 1), 'CM_Label'] = 'FN'
    return x_df
    
x_bins_test = add_col_cm_label(x_bins_test)
x_bins_train = add_col_cm_label(x_bins_train)

In [ ]:
# Plot a sample

def plot_sample(sample_id, color_case ='cyan'):
    sample_raw = x_raw_test[x_raw_test['Sample_id'] == sample_id]
    sample_bin = x_bins_test[x_bins_test['Sample_id'] == sample_id]
    sample_pred = sample_bin['Prediction'].values[0]
    sample_gt = sample_bin['Class_Label'].values[0]
    sample_cm_label = sample_bin['CM_Label'].values[0]
    
    fig, axs = plt.subplots(1, 2, figsize=(9, 4), gridspec_kw={'width_ratios': [2, 1]})
    fig.suptitle(f'Sample: {sample_id}, case: {sample_cm_label}', fontsize=16)
    
    # Plot the raw data
    axs[0].plot(sample_raw.iloc[0][:-1], color=color_case)
    axs[0].set_title(f"Raw data", fontsize=14)
    axs[0].set_xlabel("Time", fontsize=12)
    axs[0].set_ylabel("R-R interval (ms)", fontsize=12)
    axs[0].set_xticklabels([])
    axs[0].set_ylim(200, 1750)

    axs[1].barh(range(0, 31), sample_bin[bin_column_names].values[0], color=color_case)
    axs[1].set_title(f"Histogram of sample", fontsize=14)
    axs[1].set_xlabel("Count", fontsize=12)
    axs[1].set_ylabel("Bin", fontsize=12)
    axs[1].set_ylim(0, 30)
    
    plt.tight_layout()
    plt.show()
    
plot_sample(19796)


In [ ]:
# 4. Dimensionality reduction (U-MAP)
markers = ['o', 'o', 'x', 'x'] # TP, TN, FP, FN

# It seems that including both the train and test set (meaning all data) in the U-MAP doens't really change much 

# Apply UMAP on the test set
umap_model = umap.UMAP(n_neighbors=20, min_dist=0.9)#, random_state=random_seed)
sampling_percentage = 0.20
all_data_umap = umap_model.fit_transform(x_bins[bin_column_names].sample(frac=sampling_percentage, random_state=random_seed))

test_result_umap = umap_model.transform(x_bins_test[bin_column_names]) #TODO: ask if we should use the train set as well or only test set? maybe only train set to avoid data leakage?
train_result_umap = umap_model.transform(x_bins_train[bin_column_names])

x_bins_test['UMAP_1'] = test_result_umap[:, 0] 
x_bins_test['UMAP_2'] = test_result_umap[:, 1]
x_bins_train['UMAP_1'] = train_result_umap[:, 0]
x_bins_train['UMAP_2'] = train_result_umap[:, 1]

In [ ]:
#visualize the U-MAP (using all_data_umap)

plt.figure(figsize=(7, 7))
plt.title(f'U-MAP on {sampling_percentage}% of the data', fontsize=16)
plt.scatter(all_data_umap[:, 0], all_data_umap[:, 1])
plt.xlabel('UMAP 1', fontsize=14)
plt.ylabel('UMAP 2', fontsize=14)
plt.show()

In [ ]:
# Plot the U-MAP with the confusion matrix labels
fig, ax = plt.subplots(figsize=(8, 8))
fig.suptitle('U-MAP of the test set with confusion matrix labels', fontsize=16)

for i, cm_label in enumerate(['TP', 'TN', 'FP', 'FN']):
    cm_data = x_bins_test[x_bins_test['CM_Label'] == cm_label]
    ax.scatter(cm_data['UMAP_1'], cm_data['UMAP_2'], c=color_map.colors[i], marker=markers[i], label=cm_label)
    
ax.set_xlabel('UMAP 1', fontsize=14)
ax.set_ylabel('UMAP 2', fontsize=14)
ax.legend()


# plot a square box around the cluster of interest
## IMPORTANT, LOWER BOUND FIRST THEN UPPER BOUND
cluster_x = [0, 5] # U-Map 1 bounds
cluster_y = [0, 5] # U-Map 2 bounds 
box_color = 'purple'
ax.plot([cluster_x[0], cluster_x[0]], [cluster_y[0], cluster_y[1]], c=box_color)
ax.plot([cluster_x[1], cluster_x[1]], [cluster_y[0], cluster_y[1]], c=box_color)
ax.plot([cluster_x[0], cluster_x[1]], [cluster_y[0], cluster_y[0]], c=box_color)
ax.plot([cluster_x[0], cluster_x[1]], [cluster_y[1], cluster_y[1]], c=box_color)
plt.show()


In [ ]:
# Plot the U-MAP with the ground truth label
fig, ax = plt.subplots(figsize=(8, 8))
fig.suptitle('U-MAP of the test set with ground truth labels', fontsize=16)

for i, class_label in enumerate([0, 1]):
    class_data = x_bins_test[x_bins_test['Class_Label'] == class_label]
    ax.scatter(class_data['UMAP_1'], class_data['UMAP_2'], c=color_map.colors[i], label=class_label, alpha=0.5)
    
ax.set_xlabel('UMAP 1', fontsize=14)
ax.set_ylabel('UMAP 2', fontsize=14)
ax.legend()


In [ ]:
# Plot the U-MAP for the training set (optional)
fig, ax = plt.subplots(figsize=(8, 8))
fig.suptitle('U-MAP of the training set with confusion matrix labels', fontsize=16)

for i, cm_label in enumerate(['TP', 'TN', 'FP', 'FN']):
    cm_data = x_bins_train[x_bins_train['CM_Label'] == cm_label]
    ax.scatter(cm_data['UMAP_1'], cm_data['UMAP_2'], c=color_map.colors[i], marker=markers[i], label=cm_label)
    
ax.set_xlabel('UMAP 1', fontsize=14)
ax.set_ylabel('UMAP 2', fontsize=14)
ax.legend()


In [ ]:
# From inside the cluster box (in the test data plot), sample the following CM_labels
cluster_samples = x_bins_test[(x_bins_test['UMAP_1'] > cluster_x[0]) & (x_bins_test['UMAP_1'] < cluster_x[1]) & (x_bins_test['UMAP_2'] > cluster_y[0]) & (x_bins_test['UMAP_2'] < cluster_y[1])]
cluster_samples['CM_Label'].value_counts() #interesting to see the distribution of the CM labels

#color_map = mcolors.ListedColormap(['blue', 'green', 'red', 'orange']) # TP, TN, FP, FN

#sample each case from the cluster
fn_sample_id = cluster_samples[cluster_samples['CM_Label'] == 'FN'].sample(1)['Sample_id'].values[0]
tn_sample_id = cluster_samples[cluster_samples['CM_Label'] == 'TN'].sample(1)['Sample_id'].values[0]
#tp_sample_id = cluster_samples[cluster_samples['CM_Label'] == 'TP'].sample(1)['Sample_id'].values[0]
#fp_sample_id = cluster_samples[cluster_samples['CM_Label'] == 'FP'].sample(1)['Sample_id'].values[0]

plot_sample(fn_sample_id, color_case = 'red')
plot_sample(tn_sample_id, color_case = 'green')
#plot_sample(tp_sample_id, color_case = 'blue')
#plot_sample(fp_sample_id, color_case = 'orange')


N2-metric

In [ ]:
#TODO: ask teacher for follow up steps
# I can think of the following: get "distance"/difference metric between the samples of different cases in the cluster
# That is useful to know how good the dimensionality reduction is to putting similar points together

# Might as well still be AF, since doctors look at ECG, cannot derive concrete conclusion
# Use the pipeline to select qualitatively interesting samples
#CO-12 contrastivity y
# used to explain why the model does not have full accuracy
# the model is good, see confusion matrix, but visually it can be deceiving

#TODO: plot some af episodes from the raw data (the full ones, to see how these sliding windows appear)
# the following would require investigation into the raw data, by creating new dataset from TS_procces.ipynb with an extra column
#--> might reveal pauses in the same AF epsidoe
#--> might reveal that some AF episodes are not clear at all (meaning r-r interval looks normal and doctors saw something else in ecg)
#TODO: N2 as co-12! (contrastivity)
#TODO: save raw data with links to data-file names and timestamp so we can inspect the raw data

In [ ]:
def get_nearest_neighbour_distance(tree, point, exclude_point):
    point = point.reshape(1, -1)
    dist, idx = tree.query(point, k=2)

    if exclude_point:
        dist = dist[0][1]  # Take the second nearest neighbor distance
    else:
        dist = dist[0][0] # Take the nearest neighbor distance
    return dist

def get_N2(x):
    x.reset_index(drop=True, inplace=True)
    # Prepare KD-trees for both classes
    class_0_points = x[x['Class_Label'] == 0].iloc[:, :-1].values
    class_1_points = x[x['Class_Label'] == 1].iloc[:, :-1].values
    
    tree_class_0 = KDTree(class_0_points)
    tree_class_1 = KDTree(class_1_points)

    # N2 = \sum \delta(point, nearest neighbour of the same class) / \sum \delta(point, nearest neighbour of the different class)
    sum_same = 0
    sum_diff = 0

    for i in tqdm(range(len(x)), desc="Calculating N2"):
        point = x.iloc[i, :-1].values
        gt = x.iloc[i, -1]
        
        if gt == 0:
            sum_same += get_nearest_neighbour_distance(tree_class_0, point, True)
            sum_diff += get_nearest_neighbour_distance(tree_class_1, point, False)
        else:
            sum_same += get_nearest_neighbour_distance(tree_class_1, point, True)
            sum_diff += get_nearest_neighbour_distance(tree_class_0, point, False)

    N2 = sum_same / sum_diff

    return N2

In [ ]:
#calculate N2 for the U-MAP
N2_test = get_N2(x_bins_test[['UMAP_1', 'UMAP_2', 'Class_Label']])
print(f"N2 for the U-MAP of the test set: {N2_test}")

In [ ]:
#calculate N2 for the raw data
x_raw['Class_Label'] = y_raw['Class_Label']
N2_raw = get_N2(x_raw.drop(columns=['Sample_id']).sample(30000))
print(f"N2 for the raw data: {N2_raw}")

In [ ]:
# calculate N2 for binned data
N2_bins_test = get_N2(x_bins_test.drop(columns=['Sample_id', 'CM_Label', 'Prediction', 'UMAP_1', 'UMAP_2']))
print(f"N2 for the binned data of the test set: {N2_bins_test}")

In [ ]:
# RAW (40D): 0.84
# BIN (30D): 0.38
# U-MAP (2D): 0.13
#lower is better seperability of classes
# for data-centric XAI and co-12 property contrastivity, one could investigate how the N2 metric changes when improving the data quality

In [ ]:
# Calculate Fisher's Discriminant Ratio for each feature
def fisher_discriminant_ratio(df, class_col):
    features = df.columns[df.columns != class_col]
    class_labels = df[class_col].unique()

    class_1 = df[df[class_col] == class_labels[0]]
    class_2 = df[df[class_col] == class_labels[1]]

    f_ratios = []
    for feature in features:
        mu1, mu2 = class_1[feature].mean(), class_2[feature].mean()
        sigma1, sigma2 = class_1[feature].var(), class_2[feature].var()
        f_ratio = (mu1 - mu2)**2 / (sigma1 + sigma2)
        f_ratios.append(f_ratio)

    return max(f_ratios)

# Calculate the maximum Fisher's Discriminant Ratio
max_f_ratio = fisher_discriminant_ratio(x_bins_train.drop(columns=['Sample_id', 'CM_Label', 'Prediction', 'UMAP_1', 'UMAP_2']), 'Class_Label')
print("Maximum Fisher's Discriminant Ratio (F1):", max_f_ratio)

# Prototypes and Criticisms using MMD-critic

In [ ]:
!git clone https://github.com/BeenKim/MMD-critic.git

Use the functions to select prototypes and criticisms from MMD-critic/mmd.py required renaming the MMD-critic folder to MMDcritic.

Alter the x_bins or svm such that the functions also run on time-series data (that has a different format from the usps data in the MMD-critic repo and paper).

Try and try and try until it works and discuss observations before sharing with Gijs and Pooja.

In [26]:
# greedy_select_protos altered from MMD-critic/mmd.py

##############################################################################################################################
# Function choose m of all rows by MMD as per kernelfunc
# ARGS:
# K : kernel matrix
# candidate_indices : array of potential choices for selections, returned values are chosen from these  indices
# m: number of selections to be made
# is_K_sparse:  True means K is the pre-computed  csc sparse matrix? False means it is a dense matrix.
# RETURNS: subset of candidate_indices which are selected as prototypes
##############################################################################################################################

import sys

def greedy_select_protos(K, candidate_indices, m):

    K = K[:,candidate_indices][candidate_indices,:]

    n = len(candidate_indices)

    colsum = 2*np.sum(K, axis=0) / n

    selected = np.array([], dtype=int)
    for _ in range(m):
        argmax = -1
        candidates = np.setdiff1d(range(n), selected)

        s1array = colsum[candidates]
        if len(selected) > 0:
            temp = K[selected, :][:, candidates]
            
            s2array = np.sum(temp, axis=0) *2 + np.diagonal(K)[candidates]

            s2array = s2array/(len(selected) + 1)

            s1array = s1array - s2array

        else:
            s1array = s1array - (np.abs(np.diagonal(K)[candidates]))

        argmax = candidates[np.argmax(s1array)]

        selected = np.append(selected, argmax)

    return candidate_indices[selected]

In [27]:
from MMDcritic.mmd import select_criticism_regularized

num_prototypes = 10
num_criticisms = 10

support_vectors = svm.support_vectors_
print(support_vectors.shape)
selected = greedy_select_protos(
    K=support_vectors,
    # candidate_indices=np.arange(support_vectors.shape[0]),
    candidate_indices=np.arange(support_vectors.shape[0]),
    m=num_prototypes, # total number of prototypes to select
)
critselected = select_criticism_regularized(
    K=support_vectors,
    selectedprotos=selected,
    m=num_criticisms, # total number of criticisms to select
    reg='logdet',
    is_K_sparse=False
)

(12212, 31)


IndexError: index 31 is out of bounds for axis 0 with size 31

In [20]:
print(support_vectors[:10])

[[-0.08521233 -0.16382996 -0.33160387 -0.52051456 -0.69603023 -0.65208466
  -0.50582932  1.26014782  0.56272742  0.61067373 -0.10114206  0.02793971
  -0.30640557 -0.24988121 -0.20081806 -0.15616727 -0.11991584 -0.09565554
  -0.07052247 -0.06323697 -0.05538276 -0.04548793 -0.05529552 -0.05294446
  -0.04437971 -0.03752698 -0.03196376 -0.01970675 -0.02922586 -0.02487771
  -0.02394048]
 [-0.08521233 -0.16382996 -0.33160387 -0.52051456 -0.69603023 -0.65208466
  -0.50582932 -0.47353519 -0.28763072 -0.27423017  0.94202488  2.19391293
   0.20559993  0.18503031 -0.20081806 -0.15616727 -0.11991584 -0.09565554
  -0.07052247 -0.06323697 -0.05538276 -0.04548793 -0.05529552 -0.05294446
  -0.04437971 -0.03752698 -0.03196376 -0.01970675 -0.02922586 -0.02487771
  -0.02394048]
 [-0.08521233 -0.16382996 -0.33160387 -0.52051456 -0.34677157  1.69989178
   0.14145954  0.39330632 -0.28763072 -0.16361718  0.24658025 -0.10743362
  -0.30640557 -0.03242545  0.07515918 -0.15616727 -0.11991584 -0.09565554
  -0.070